<div style="background-color:#000;"><img src="pqn.png"></img></div><div><a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://www.pyquantnews.com/getting-started-with-python-for-quant-finance/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.</div>

## Library installation
Install the libraries needed to download market data, compute vectorized metrics, and render plots so the notebook runs end to end.

In [ ]:
!pip install yfinance pandas numpy matplotlib

yfinance gives us quick access to historical prices, pandas handles time series, numpy powers fast array ops, and matplotlib lets pandas render the underwater charts. Installing them up front avoids environment surprises when we get to plotting and rolling calculations.

## Imports and setup
We use yfinance to download SPY price history for the study period, and numpy for vectorized operations used inside the drawdown calculations.

In [ ]:
import yfinance as yf
import numpy as np

Keeping imports minimal mirrors production pipelines where dependencies are kept lean and predictable. yfinance is fine for tutorials; in production you would typically swap in a data vendor but keep the same numpy-based logic.

## Fetch price data and returns
Pull SPY daily prices for the sample window and transform them into simple daily returns, which are the input to our drawdown functions.

In [ ]:
data = yf.download("SPY", start="2020-01-01", end="2022-07-31")
returns = data.Close.pct_change()

Using adjusted close aligns with what we care about: investable returns including dividends and splits. pct_change introduces a leading NaN, which is expected and handled later; the important part is we’re creating a clean, noncumulative return stream that we can roll up into an equity curve.

## Define drawdown utilities and metrics
Implement a vectorized drawdown series and a companion max drawdown function so we can reuse the same logic in backtests, dashboards, and rolling risk checks.

In [ ]:
def drawdown(returns):
    """Determines the drawdown

    Parameters
    ----------
    returns : pd.Series
        Daily returns of an asset, noncumulative

    Returns
    -------
    drawdown : pd.Series
    """
    returns.fillna(0.0, inplace=True)

    cumulative = (returns + 1).cumprod()

    running_max = np.maximum.accumulate(cumulative)

    return (cumulative - running_max) / running_max

In [ ]:
def max_drawdown(returns):
    """Determines the maximum drawdown

    Parameters
    ----------
    returns : pd.Series
        Daily returns of an asset, noncumulative

    Returns
    -------
    max_drawdown : float
    """
    return np.min(drawdown(returns))

The drawdown series tracks how far underwater we are relative to the running peak, which is how pros experience risk day to day. Filling NaNs with zero avoids contaminating the cumulative product at the start; note this mutates the input Series in place, so pass a copy if you need the original untouched. The minimum of the underwater series is the max drawdown, the single pain number allocators and risk teams anchor on.

## Plot underwater and rolling risk
Visualize the full-sample underwater curve, then a 30-day rolling max drawdown to spot regime shifts and trigger de-risking rules earlier.

In [ ]:
drawdown(returns).plot(kind="area", color="salmon", alpha=0.5)

In [ ]:
returns.rolling(30).apply(max_drawdown).plot(
    kind="area",
    color="salmon",
    alpha=0.5,
)

The underwater area chart shows the depth and duration of capital losses, which is more decision-relevant than average return when markets stress. Rolling max drawdown highlights localized pockets of pain that get averaged away in full-sample stats, making it a practical alert for position sizing and pause rules. In production, this series feeds a dashboard and simple thresholds to keep risk within cash and mandate limits.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://www.pyquantnews.com/getting-started-with-python-for-quant-finance/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.